In [26]:
import numpy as np
import os
import gym
import zipfile
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from imitation.algorithms import bc
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

rng = np.random.default_rng(0)

In [31]:
def load_data(directory):
    states, actions = [], []
    for filename in os.listdir(directory):
        print(filename)
        if filename.endswith('.npz'):
            filepath = os.path.join(directory, filename)
            try:
                data = np.load(filepath, allow_pickle=True)  # ensure allow_pickle is set appropriately
                states.append(data['arr_0'])  # ensure key matches what's in your files
                actions.append(data['arr_1'])  # ensure key matches what's in your files
            except EOFError:
                print(f"Failed to load data from {filepath}, the file may be corrupted.")
            except KeyError as e:
                print(f"Key error {e} in file {filepath}, check the array keys.")
    return np.concatenate(states), np.concatenate(actions)

In [16]:
#sb3 only wants to work with gymnasium and not gym, install shimmy
#pip install shimmy>=0.2.1
env = gym.make('SuperMarioBros-v3')
env = DummyVecEnv([lambda:env])

In [33]:
directory = './recordings_1'
states, actions = load_data(directory)
transitions = {'obs': states, 'acts': actions, 'infos': []}

imitation_mario_rec_Carson_032624_140806.npz
imitation_mario_rec_Carson_032624_141113.npz
imitation_mario_rec_carson_032824_184208.npz
Failed to load data from ./recordings_1/imitation_mario_rec_carson_032824_184208.npz, the file may be corrupted.
imitation_mario_rec_carson_032824_184707.npz
Failed to load data from ./recordings_1/imitation_mario_rec_carson_032824_184707.npz, the file may be corrupted.
imitation_mario_rec_carson_032124_145619.npz
imitation_mario_rec_Carson_032624_140935.npz
imitation_mario_rec_carson_032824_190019.npz
imitation_mario_rec_carson_032124_145652.npz
imitation_mario_rec_carson_032124_143622.npz
imitation_mario_rec_carson_032124_225559.npz
imitation_mario_rec_carson_032124_143819.npz
imitation_mario_rec_carson_032124_144528.npz
imitation_mario_rec_Carson_032624_140635.npz
imitation_mario_rec_Carson_032624_141236.npz
imitation_mario_rec_Carson_032124_230019.npz
imitation_mario_rec_Carson_032624_141422.npz
imitation_mario_rec_carson_032824_185710.npz
imitation

In [36]:
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng = rng,
)

In [49]:
print (type(transitions['obs']))
print ((transitions['obs'].shape))
print ((transitions['acts'].shape))

<class 'numpy.ndarray'>
(153537, 240, 256)
(153537,)


In [43]:
bc_trainer.train(n_epochs=1)

reward, _ = evaluate_policy(bc_trainer.policy, env, n_eval_episodes=10)
print("Reward:", reward)

0batch [00:00, ?batch/s]


TypeError: string indices must be integers, not 'str'